<h1>Modeling testbook</h1>

In [2]:
## Latest and greatest sklearn
# !pip install sklearn --upgrade
## Dependencies for data preprocessing
import pandas as pd
from datetime import datetime as dt
import numpy as np
import requests
#import config
import json
import time
import datetime
from pprint import pprint
# from nba_api.stats.endpoints import leaguegamelog
# from nba_api.stats.endpoints import boxscoresummaryv2
import pandas
import sqlite3

In [3]:
## Connect to sqlite db
conn = sqlite3.connect("../../NBAapi/Persistence/app.db")
## Teams cheatsheet
## Warriors NBA_Id: 1610612744, TeamId: 10
## Celtics NBA_Id: 1610612738, TeamId: 2
## Lakers NBA_Id: 1610612747, TeamId: 14
TeamName = 'Lakers'
c = conn.cursor()
c.execute("SELECT TeamId, NBA_API_TeamId FROM Team WHERE Name = '"+TeamName+"'")
## Fetching team values from the table
## Also sets all values to be used for insertion into Predicion/Summary
teamDf = c.fetchone()
TeamId = teamDf[0]
NBA_API_TeamId = teamDf[1]
Features = 6
# print(TeamId)
# print(NBA_API_TeamId)


In [4]:

## Load boxscore data into a DataFrame
## First load data from Team playing from home
home_games = "SELECT PointsHome AS Points, FGPctHome AS FGPct, FTPctHome AS FTPct, FG3PctHome AS FG3Pct, ASTHome AS AST, REBHome AS REB, HomeTeamWin AS Win "
home_games += "FROM GameBoxScore WHERE NBA_API_HomeTeamId = "+str(NBA_API_TeamId)+" ORDER BY NBA_API_GameId DESC"
df_home = pd.read_sql_query(home_games, conn)
## Then load data from away games
away_games = "SELECT PointsAway AS Points, FGPctAway AS FGPct, FTPctAway AS FTPct, FG3PctAway AS FG3Pct, ASTAway AS AST, REBAway AS REB, AwayTeamWin AS Win "
away_games += "FROM GameBoxScore WHERE NBA_API_AwayTeamId = "+str(NBA_API_TeamId)+" ORDER BY NBA_API_GameId DESC"
df_away = pd.read_sql_query(away_games, conn)
## Resetting index for both dataframes, conca/join/merge would fail without this (duplicates).
# df_home = df_home.loc[~df_home.index.duplicated(keep='first')]
# df_away = df_away.loc[~df_away.index.duplicated(keep='first')]
## Stack the DataFrames on top of each other
## This one doesn't work;; df = df_home.concat(df_away, how='left')
## Concat home and away games
df = pd.concat([df_home, df_away])
## May need to drop last column of a dataframe
# df = df.iloc[: , :-1]
## Resets dataframe index, SKlern may throw ValueError otherwise
# df = df.reset_index()
# df['Win'].to_numpy('int32')
df.head()

,Points,FGPct,FTPct,FG3Pct,AST,REB,Win
0,103,0.407,0.760,0.323,27,49,1
1,100,0.418,0.875,0.286,21,35,0
2,92,0.395,0.714,0.325,19,50,0
3,109,0.442,0.867,0.250,19,51,1
4,108,0.463,0.857,0.368,21,41,0


In [5]:
# conn.close()
# grab every stat except the 'win' column as your X features
X = df.drop('Win', axis=1)
print(X.shape)
X.head()

(230, 6)


,Points,FGPct,FTPct,FG3Pct,AST,REB
0,103,0.407,0.760,0.323,27,49
1,100,0.418,0.875,0.286,21,35
2,92,0.395,0.714,0.325,19,50
3,109,0.442,0.867,0.250,19,51
4,108,0.463,0.857,0.368,21,41


In [11]:
# Set our Wins/Ys
y = df['Win']
y.head()

0    1
1    0
2    0
3    1
4    0
Name: Win, dtype: int64

In [12]:
# Import training and scaling tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [13]:
## Split your Xs and Ys (default split 72/25)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
## Scale X data to keep value magnitude similar and mitigate possible high variance 
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

<h1>Random Forest (using sklearn)</h1>

In [15]:
# Lets try 2 different RFC with 200 and 400 estimators, and select the best from both
from sklearn.ensemble import RandomForestClassifier
# Model definition for model #1
modelRF1 = RandomForestClassifier(n_estimators=10)
# Fit our training data
modelRF1.fit(X_train_scaled, y_train)
# Model definition for model #1
modelRF2 = RandomForestClassifier(n_estimators=200)
# Fit model #2
modelRF2.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [16]:
testScore1 = modelRF1.score(X_test_scaled, y_test)
testScore2 = modelRF2.score(X_test_scaled, y_test)
# Check RFC #1 results
print(f"Model #1 Training score: {round(modelRF1.score(X_train_scaled, y_train)*100,2)}%")
print(f"Model #1 Test score: {round(testScore1*100,2)}%")
# Check RFC #2 results
print(f"Model #2 Training score: {round(modelRF2.score(X_train_scaled, y_train)*100,2)}%")
print(f"Model #2 Test score: {round(testScore2*100,2)}%")
#Select best model
if testScore1 >= testScore2:
    modelRF = modelRF1
    print('Selected model #1')
else:
    modelRF = modelRF2
    print('Selected model #2')
bestScore = modelRF.score(X_test_scaled, y_test)

Model #1 Training score: 100.0%
Model #1 Test score: 70.69%
Model #2 Training score: 100.0%
Model #2 Test score: 72.41%
Selected model #2


In [18]:
## Dump predictions into a LabelEncoder
## Compare predictions against actual game results
result_encoder = LabelEncoder()
result_encoder.fit(y_train)
encoded_test_results = modelRF.predict(X_test_scaled)
results_labels = result_encoder.inverse_transform(encoded_test_results)
print(f"7 total Predictions:   {results_labels[:7]}")
print(f"7 total Actual results: {y_test[:7].tolist()}")

7 total Predictions:   [0 1 1 1 0 1 0]
7 total Actual results: [0, 1, 1, 1, 0, 1, 0]


<h3>Random Forrest gets us to 75% accuracy predicting win/loss based on team stats per game - not bad at all!</h3>

<h1>Neural network (using keras)</h1>

In [19]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
## Convert Ys to one-hot encode
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [20]:
## Dependencies for Sequential model and Dense for the hidden layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [21]:
# Fill our model with Dense layers, using Relu activation (not used at this moment but it allows backpropagation)
# input_dim set to 6 because we have 6 X features
# Last layer set to softmax (we want a soft result, get it?) as we look for binary classification 
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=6))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [22]:
## Fit model, adam optimizer is straightforward to implement with MLnet (see roadmap)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
## Summary, very important
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               700       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 21,102
Trainable params: 21,102
Non-trainable params: 0
_________________________________________________________________


In [24]:
## Training time, set to 100 epochs
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
172/172 - 0s - loss: 0.6624 - acc: 0.5814
Epoch 2/100
172/172 - 0s - loss: 0.6332 - acc: 0.6047
Epoch 3/100
172/172 - 0s - loss: 0.6256 - acc: 0.6047
Epoch 4/100
172/172 - 0s - loss: 0.6151 - acc: 0.6221
Epoch 5/100
172/172 - 0s - loss: 0.6087 - acc: 0.6453
Epoch 6/100
172/172 - 0s - loss: 0.5979 - acc: 0.6395
Epoch 7/100
172/172 - 0s - loss: 0.5883 - acc: 0.6802
Epoch 8/100
172/172 - 0s - loss: 0.5767 - acc: 0.6977
Epoch 9/100
172/172 - 0s - loss: 0.5663 - acc: 0.6977
Epoch 10/100
172/172 - 0s - loss: 0.5581 - acc: 0.7209
Epoch 11/100
172/172 - 0s - loss: 0.5515 - acc: 0.7384
Epoch 12/100
172/172 - 0s - loss: 0.5478 - acc: 0.7267
Epoch 13/100
172/172 - 0s - loss: 0.5371 - acc: 0.7209
Epoch 14/100
172/172 - 0s - loss: 0.5315 - acc: 0.7442
Epoch 15/100
172/172 - 0s - loss: 0.5265 - acc: 0.7209
Epoch 16/100
172/172 - 0s - loss: 0.5177 - acc: 0.7558
Epoch 17/100
172/172 - 0s - loss: 0.5286 - acc: 0.7442
Epoch 18/100
172/172 - 0s - loss: 0.5093 - acc: 0.7326
Epoch 19/100
172/17

In [25]:
## Check results (drum roll)
modelLoss, modelAccuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Neural Network - Loss: {modelLoss}, Accuracy: {modelAccuracy}")

58/58 - 0s - loss: 0.5608 - acc: 0.7069
Neural Network - Loss: 0.560757570225617, Accuracy: 0.7068965435028076


In [26]:
## Compare predictions against actual game results
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [27]:
print(f"7 total Predictions:   {prediction_labels[:7]}")
print(f"7 total Actual results: {y_test[:7].tolist()}")

7 total Predictions:   [0 1 1 1 0 1 0]
7 total Actual results: [0, 1, 1, 1, 0, 1, 0]


In [98]:
## ADD PREDICTIONS TO SQLITE DB
import sqlite3
from sqlite3 import Error
import pandas
from datetime import datetime

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def insert_predictionsummary(conn, summary):
    """
    Inserts Prediction Summary
    """
    sql = ''' INSERT INTO SeriesPredictionSummary
              (DateAdded, Features, Loss, Accuracy, Recall, TrainSize, TestSize, Epochs, Technique, Comments, TeamId, NBA_API_TeamId)
              VALUES(?,?,?,?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, summary)
    conn.commit()
    return cur.lastrowid
    
def insert_predictions(conn, prediction):
    """
    Inserts Single Prediction, detail of Summary
    """
    sql = ''' INSERT INTO SinglePrediction
              (TeamId, NBA_API_TeamId, Label, Win, SeriesPredictionSummaryId)
              VALUES(?,?,?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, prediction)
    conn.commit()

def main():
    # database = "../../NBAapi/Persistence/app.db"
    database = "../../NBAapi/NBAapi/app.db"
    # create a database connection
    conn = create_connection(database)
    with conn:
        seriesId = insert_predictionsummary(conn, (datetime.now(), 6, modelLoss, modelAccuracy.astype(str), 0, X_train.shape[0], X_test.shape[0], 100, "CNN", "No comments", TeamId, NBA_API_TeamId))
        for x in prediction_labels[:7]:
            insert_predictions(conn, (TeamId, NBA_API_TeamId, '', x.astype(str), seriesId))
    print(f'All data inserted to SeriesPredictionSummaryId: {seriesId}')
    conn.close()
if __name__ == '__main__':
    main()

All data inserted to SeriesPredictionSummaryId: 1
